In [1]:
import pandas as pd

In [37]:
import pandas as pd

# Load the DataFrame
df_parl = pd.read_csv('data/wahl70_sprengel.csv', delimiter=';', encoding='ISO-8859-1', skiprows=1)

# Filter the columns to get 'Sprengel_Nr', all 'Prozente' and 'Name/Wählergruppe' columns
columns_of_interest = ['Sprengel_Nr'] + [col for col in df_parl.columns if 'Prozente' in col or 'Name/Wählergruppe' in col]
simplified_df = df_parl[columns_of_interest]

# To ensure all wählergruppen are included, determine the maximum index from column names
max_wahlergruppe_index = max(
    int(col.split('Wählergruppe')[-1]) 
    for col in df_parl.columns 
    if 'Wählergruppe' in col and any(char.isdigit() for char in col)
)

# Rename the 'Prozente' columns to reflect the corresponding party names and then drop 'Name/Wählergruppe' columns
name_columns_to_drop = []
for i in range(1, max_wahlergruppe_index + 1):
    percent_col = f'Prozente/Wählergruppe{i}'
    name_col = f'Name/Wählergruppe{i}'
    # Ensure that both columns exist before renaming and planning to drop
    if percent_col in simplified_df.columns and name_col in simplified_df.columns:
        party_name = simplified_df[name_col].iloc[0]  # Assuming the first row gives the party name
        simplified_df.rename(columns={percent_col: party_name}, inplace=True)
        name_columns_to_drop.append(name_col)

# Drop the 'Name/Wählergruppe' columns
simplified_df.drop(columns=name_columns_to_drop, inplace=True)

# Convert all percentage columns from string to float, replacing commas with dots
for col in simplified_df.columns:
    if col != 'Sprengel_Nr':  # Skip the 'Sprengel_Nr' column
        simplified_df[col] = simplified_df[col].str.replace(',', '.').astype(float)

# Create 'Gewinner' column by finding the party with the highest percentage per row
simplified_df['Gewinner'] = simplified_df.drop(['Sprengel_Nr'], axis=1).idxmax(axis=1)

# First, handle NaN values in 'Sprengel_Nr' and convert to integer
simplified_df['Sprengel_Nr'] = pd.to_numeric(simplified_df['Sprengel_Nr'], errors='coerce').fillna(0).astype(int)

# Save the DataFrame to a CSV file with UTF-8 encoding
simplified_df.to_csv('data/ergebnis_partei.csv', index=False, encoding='ISO-8859-1')

/tmp/ipykernel_37163/2523471326.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simplified_df.rename(columns={percent_col: party_name}, inplace=True)
/tmp/ipykernel_37163/2523471326.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simplified_df.drop(columns=name_columns_to_drop, inplace=True)
/tmp/ipykernel_37163/2523471326.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c